In [8]:
%matplotlib notebook

import pickle
import sys
import numpy as np
import import_ipynb
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.sparse.linalg import eigsh
from IPython.display import SVG, display
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from matplotlib.animation import FuncAnimation

# Adicionar o diretório raiz do projeto ao sys.path
project_root = Path().resolve().parent  
sys.path.append(str(project_root))
print("Project root added to sys.path:", project_root)

# Importando notebooks diretamente
try:
    import problem_statement as ps  
    from fem_processing import vectorial_matrices_assembly as assembly
    from fem_processing import master_domain as master
    from fem_pos_processing import graph_results as graph
    print("Modules imports were successful!")
except ModuleNotFoundError as e:
    print(f"Modules were not found: {e}")
except ImportError as e:
    print(f"Error in import: {e}")

Project root added to sys.path: C:\Users\adilt\OneDrive\05_GIT\PPGEE-2024.2-MEF
Modules imports were successful!


# Project 5: Rectangular Guide Cavity
# 6-edge Nedelec Tetrahedron

In [9]:
FINITE_ELEMENT = ("Tetrahedron", 1)
BOUNDARY = [{'tag': 201, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}]
MATERIAL = [{'tag': 301, 'name': 'free_space', 'ur': 1, 'er': 1}]

# Create mesh from file geometry
mesh_data = ps.create_rectangular_cavity(FINITE_ELEMENT, BOUNDARY, MATERIAL, h=0.1, view_mesh=False)

# Applying additional physical properties to the mesh
mesh_data = ps.apply_physics(mesh_data)

Malha salva em pre_processing/mesh/rectangular_cavity_domain_Tetrahedron1.msh
Malha salva em pre_processing/mesh/rectangular_cavity_domain_Tetrahedron1.vtk
Model rectangular_cavity (3D)
Info     : 27 geometric entities
Info     : 2 physical groups
Info     : 396 nodes in total
Info     : 2024 edges in total
Info     : 1311 3-D elements in total


In [10]:
dirichlet_edges = {key: edge for key, edge in mesh_data['edges'].items() if edge['bc']['tag'] == 201}

# FEM Solution
## `assembly.local_matrices()`

In [11]:
import numpy as np
from scipy.sparse import lil_matrix
from fem_processing import gaussian_quadrature
from fem_processing import master_domain

# Arestas livres
free_edges = {key: value
                for key, value in mesh_data['edges'].items() if value['bc']['type'] != 'Dirichlet'}

# Mapeamento de índices globais para índices reduzidos
global_to_reduced = {global_edge: idx
                        for idx, global_edge in enumerate(free_edges.keys())}

# Initialize the global matrix and vector
Sgr = lil_matrix((len(free_edges), len(free_edges)), dtype='complex128')
Mgr = lil_matrix((len(free_edges), len(free_edges)), dtype='complex128')
bgr = lil_matrix((len(free_edges), 1), dtype='complex128')

## `assembly.local_matrices()`

In [12]:
for cell in mesh_data['cell'].values():
    Ne = len(cell['conn_edge'])
    Se = np.zeros((Ne, Ne), dtype='complex128') # Matriz de massa do elemento
    Me = np.zeros((Ne, Ne), dtype='complex128') # Matriz de massa do elemento
    be = np.zeros((Ne, 1), dtype='complex128')  # Vetor de carga do elemento

    # Define the Finite Element Type
    ElementType, ElementOrder = FINITE_ELEMENT

    # Material properties
    pe = cell['p(x)']
    qe = cell['q(x)']
    fe = cell['f(x)']

    # Cálculo da matriz Jacobiana
    Je = cell['geo']['jacobian']
    Jdet, JinvT = np.abs(np.linalg.det(Je)), np.linalg.inv(Je).T  

    # Rotacional das funções de forma
    curl_phi_hat = [
        np.array([[ 0], [-2], [ 2]]),
        np.array([[ 2], [ 0], [-2]]),
        np.array([[-2], [ 2], [ 0]]),
        np.array([[ 0], [ 0], [ 2]]),
        np.array([[ 0], [-2], [ 0]]),
        np.array([[ 2], [ 0], [ 0]])
    ]

    # Tensor métrico contravariante
    Gn = JinvT.T @ JinvT

    # Get the Gauss points and weights
    gauss_points, gauss_weights = gaussian_quadrature.gauss_data(FINITE_ELEMENT)
    for xik, wk in zip(gauss_points, gauss_weights):
        # Função de forma vetorial no ponto de integração
        if ElementType == 'Triangle':
            phi_hat = master_domain.shape_functions_nedelec_3d(*xik)
        elif ElementType == 'Tetrahedron':
            phi_hat = master_domain.shape_functions_nedelec_tetrahedra(*xik)

        for i in range(Ne):
            # Transformação de Piola Covariante
            phi_i = JinvT @ phi_hat[i]
            curl_phi_i = 1/Jdet * Je @ curl_phi_hat[i]

            # Vetor de cargas
            be[i] += (fe.T @ phi_i).item() * Jdet * wk
            
            for j in range(Ne):
                # Transformação de Piola Covariante
                phi_j = JinvT @ phi_hat[j]
                curl_phi_j = 1/Jdet * Je @ curl_phi_hat[j]

                # Matriz de rigidez
                Se[i, j] += (curl_phi_i.T @ curl_phi_j).item() * Jdet * wk

                # Matriz de massa
                Me[i, j] += (phi_hat[i].T @ Gn @ phi_hat[j]).item() * Jdet * wk
    
    # Início do processo de montagem do sistema global
    for i, ig in enumerate(cell['conn_edge']):
        if ig in free_edges:
            ig_red = global_to_reduced[ig]     
                               
            for j, jg in enumerate(cell['conn_edge']):
                if jg in free_edges:
                    jg_red = global_to_reduced[jg]
                    Sgr[ig_red, jg_red] += Se[i, j]
                    Mgr[ig_red, jg_red] += Me[i, j]

                # A aresta global é de Dirichlet
                else:
                    # Contribuição de Dirichlet para o vetor reduzido
                    bgr[ig_red, 0] += -(Se[i, j] + Me[i, j]) * mesh_data['edges'][jg]['bc']['value']
            
            # preenche o vetor global de carga
            bgr[ig_red, 0] += be[i]

## Eigenvalue Problem

In [15]:
# Resolver o problema de autovalor generalizado
# k Número de autovalores/autovetores desejados
eigenvalues, eigenvectors = eigsh(Sgr, k=30, M=Mgr, which='SM')

# Definir um limiar para eliminar erros de truncamento
threshold = 1e-8
positive_indices = np.where(eigenvalues > threshold)[0]

# Filtrar autovalores positivos e ordenar
sp_eigenvalues = eigenvalues[positive_indices]
sp_eigenvectors = eigenvectors[:, positive_indices]

# Ordenar os autovalores positivos e ajustar os autovetores na mesma ordem
s_indices = np.argsort(sp_eigenvalues)
sp_eigenvalues = sp_eigenvalues[s_indices]
sp_eigenvectors = sp_eigenvectors[:, s_indices]

print("Eigenvector shape:", eigenvectors.shape)
graph.matrix2table(np.sqrt(sp_eigenvalues), title='Menores autovalores positivos')

Eigenvector shape: (1070, 30)


,1
1,6.074e+00
2,8.094e+00
3,8.406e+00
4,9.294e+00
5,9.817e+00
6,9.867e+00
7,9.945e+00
8,1.061e+01
9,1.072e+01
10,1.120e+01


Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2025.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.